In [5]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import parse_nums
from src.perceptron import MultilayerPerceptron

PATH = "./data/Ej3B-digitos.txt"

OUTPUT_ACTIVATION = ["TANH", "LOG"]
HIDDEN_ACTIVATION = ["TANH", "LOG"]
BETA = 1

TRAINING_TYPE = "PERCENTAGE"
K_FOLD = 4
LR = 0.1
BIAS = 1
EPOCHS = 10000
MIN_ERROR = 0.01
TRAINING_PERCENTAGE = 0.9

QTY_HIDDEN_LAYERS = 2
QTY_NODES_IN_HIDDEN_LAYERS = [16, 10]

OPTIMIZER_METHOD = ["MOMENTUM", "ADAM"]
K_FOLD = 4
ALPHA = 0.8
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8

input_data, expected_data = parse_nums(PATH, 7, 2)

#-------------------------------------------------
# VARIANDO EL LEARNING RATE
#-------------------------------------------------

arr_of_errors = []
arr_of_epochs = []
lrs = [round(LR*(10**(-i)),4) for i in range(4)]

for i, opt_method in enumerate(OPTIMIZER_METHOD):
    arr_of_error_aux = []
    arr_of_epochs_aux = []
    for out_act in OUTPUT_ACTIVATION:
        arr_of_error_aux2 = []
        arr_of_epochs_aux2 = []
        for hidden_act in HIDDEN_ACTIVATION:
            arr_of_error_aux3 = []
            arr_of_epochs_aux3 = []
            for lr in lrs:
                perceptron = MultilayerPerceptron(input_data, expected_data, lr, BIAS,
                                                EPOCHS, TRAINING_TYPE, TRAINING_PERCENTAGE, K_FOLD, MIN_ERROR,
                                                QTY_HIDDEN_LAYERS, QTY_NODES_IN_HIDDEN_LAYERS, 
                                                out_act, hidden_act, BETA,
                                                opt_method, ALPHA, BETA1, BETA2, EPSILON, [-1,1])
                mse_errors, total_epochs, acurracy, test_mse = perceptron.train()
                arr_of_error_aux3.append(mse_errors)
                arr_of_epochs_aux3.append(total_epochs)

            arr_of_error_aux2.append(arr_of_error_aux3)
            arr_of_epochs_aux2.append(arr_of_epochs_aux3)

        arr_of_error_aux.append(arr_of_error_aux2)
        arr_of_epochs_aux.append(arr_of_epochs_aux2)

    arr_of_errors.append(arr_of_error_aux)
    arr_of_epochs.append(arr_of_epochs_aux)
    
colors = ['blue', 'green', 'red', 'orange']

for i, opt_method in enumerate(OPTIMIZER_METHOD):
    for j, out_act in enumerate(OUTPUT_ACTIVATION):
        for k, hidden_act in enumerate(HIDDEN_ACTIVATION):
            for s in range(len(arr_of_errors[i][j][k])):
                plt.plot(range(arr_of_epochs[i][j][k][s]), arr_of_errors[i][j][k][s], color=colors[s], label=f"η={lrs[s]}")
            plt.title(f'Perceptron Multicapa con optimizacion {opt_method} \n output={out_act}, hidden={hidden_act}')
            plt.xlabel("Epocas")
            plt.ylabel("Error (MSE)")
            plt.legend(loc='best', bbox_to_anchor=(1.2, 1.0))
            plt.show()
       

0
Finished Training. 
 MSE: 0.0009651833164739233
Aca
-0.5812114145510816
Test Acurracy: 0.0
Test MSE = 2.5002295375066326
0
Finished Training. 
 MSE: 0.009433644309179257
Aca
0.913230945671125
Test Acurracy: 0.0
Test MSE = 3.660452651473627
0
Finished Training. 
 MSE: 0.009942603537100803
Aca
-0.09933455770252875
Test Acurracy: 1.0
Test MSE = 0.8111982389488995
0
1000
2000
Finished Training. 
 MSE: 0.009995618492881279
Aca
-0.5567941055341911
Test Acurracy: 0.0
Test MSE = 2.4236078870260025
0
Finished Training. 
 MSE: 0.0014715866392177182
Aca
-0.9929821695802092
Test Acurracy: 1.0
Test MSE = 4.924994380094056e-05
0
Finished Training. 
 MSE: 0.009773262688227466
Aca
-0.8593527453455359
Test Acurracy: 0.0
Test MSE = 3.457192631623981
0
1000
Finished Training. 
 MSE: 0.00998191750484046
Aca
0.6158498384091025
Test Acurracy: 1.0
Test MSE = 0.14757134665031268
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Finished Training. 
 MSE: 0.009997953957878218
Aca
-0.6350212757254897
Test Acurrac

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import parse_nums
from src.perceptron import MultilayerPerceptron

#-------------------------------------------------
# VARIANDO EL PORCENTAJE DE ENTRENAMIENTO
#-------------------------------------------------

PATH = "./data/Ej3B-digitos.txt"
OUTPUT_ACTIVATION = "TANH"
HIDDEN_ACTIVATION = "TANH"
BETA = 1.0

TRAINING_TYPE = "PERCENTAGE"
LR = 0.001
BIAS = 1
EPOCHS = 50000
MIN_ERROR = 0.01

QTY_HIDDEN_LAYERS = 2
QTY_NODES_IN_HIDDEN_LAYERS = [16, 10]

OPTIMIZER_METHOD = ["MOMENTUM", "ADAM"]
K_FOLD = 4
ALPHA = 0.8
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
TRAIN_PERCENTAGES = [round(0.1*i,2) for i in range(4,11)]
ITERATIONS = 50

input_data, expected_data = parse_nums(PATH, 7, 2)

arr_avg_train_mses = []
arr_std_train_mses = []
arr_avg_test_mses = []
arr_std_test_mses = []

for i, opt_method in enumerate(OPTIMIZER_METHOD):
    avg_train_mses = []
    std_train_mses = []
    avg_test_mses = []
    std_test_mses = []
    for p in TRAIN_PERCENTAGES:
        train = []
        test = []
        for iter in range(ITERATIONS):
            perceptron = MultilayerPerceptron(input_data, expected_data, LR, BIAS,
                                                EPOCHS, TRAINING_TYPE, p, K_FOLD, MIN_ERROR,
                                                QTY_HIDDEN_LAYERS, QTY_NODES_IN_HIDDEN_LAYERS, 
                                                OUTPUT_ACTIVATION, HIDDEN_ACTIVATION, BETA,
                                                opt_method, ALPHA, BETA1, BETA2, EPSILON,  [-1,1])
            mse_errors, total_epochs, test_acurracy, test_mse = perceptron.train()
            #Testeo con el que entrene
            print(f"-------------- Entrenamiento con p={p}----------------")
            if(p == 1):
                train_acurracy, train_mse = perceptron.test(perceptron.input_data, perceptron.expected_data)
            else:
                train_acurracy, train_mse = perceptron.test(perceptron.train_input_data, perceptron.train_expected_data)
            print("------------------------------")
            train.append(train_acurracy)
            test.append(test_acurracy)
            
        train_aux = sum(train) / len(train)
        avg_train_mses.append(train_aux)
        error_train = np.std(train) / np.sqrt((len(train)))
        std_train_mses.append(error_train)
        
        test_aux = sum(test) / len(test)
        avg_test_mses.append(test_aux)
        error_test = np.std(test) / np.sqrt((len(test)))
        std_test_mses.append(error_test)
                    
    arr_avg_train_mses.append(avg_train_mses)
    arr_std_train_mses.append(std_train_mses)
    arr_avg_test_mses.append(avg_test_mses)
    arr_std_test_mses.append(std_test_mses)
    
for i, opt_method in enumerate(OPTIMIZER_METHOD):
    plt.errorbar(TRAIN_PERCENTAGES, arr_avg_train_mses[i], yerr=arr_std_train_mses[i], color="blue", capsize=4, label="Train")
    plt.errorbar(TRAIN_PERCENTAGES, arr_avg_test_mses[i], yerr=arr_std_test_mses[i], color="red", capsize=4, label="Test")
    plt.title(f'Perceptron Multicapa con optimizacion {opt_method} \n output={OUTPUT_ACTIVATION}, hidden={HIDDEN_ACTIVATION}')
    plt.xlabel("Porcentaje de entrenamiento")
    plt.ylabel("Acurracy")
    plt.legend(loc='best', bbox_to_anchor=(1.2, 1.0))
    plt.show()

KeyboardInterrupt: 